<a href="https://colab.research.google.com/github/tikendraw/language-translation-model/blob/main/language_translation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Language Translation Model (English to Hindi)


---




In [1]:
import os, sys
if 'google.colab' in sys.modules:
    
    ! git clone https://github.com/tikendraw/language-translation-model.git 
    os.chdir('language-translation-model') 
    print(os.getcwd())

In [2]:
# Dependencies
# ! pip install polars -q
import polars as pd
from datetime import datetime
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model , optimizers
from tensorflow.keras.layers import Attention, LSTM, Bidirectional, Dense, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, AveragePooling1D, Dropout, concatenate, Concatenate

import seaborn as sns
import matplotlib.pyplot as plt

# !pip install tensorflow_hub
# import tensorflow_hub as hub
import os
import re
import json
from sklearn.model_selection import train_test_split

print('Tf version: ',tf.__version__)
print('GPU: ', is_gpu:=len(tf.config.list_physical_devices('GPU')))
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ["TFHUB_CACHE_DIR"] = './tmp/tfhub'

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
None
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""
if is_gpu:
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
    config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(physical_devices)


2023-02-20 12:01:42.813144: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 12:01:44.274116: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/t/miniconda3/envs/tf_new/lib/
2023-02-20 12:01:44.274217: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/t/miniconda3/envs/tf_new/lib/
2023-02-20 12:01:44.274226: W tensorflow/compiler/tf2ten

Tf version:  2.11.0
GPU:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-02-20 12:01:46.901052: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-20 12:01:46.914178: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-20 12:01:46.914381: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
dataset_url = 'http://www.manythings.org/anki/hin-eng.zip'

In [4]:
# # Download the dataset
if 'google.colab' in sys.modules:
    # donwload
    !wget $dataset_url -P dataset

    # # Unzip the downloaded file
    !unzip ./dataset/hin-eng.zip -d ./dataset

    # # Show size
    !du -h  ./dataset

# Reading the data

In [5]:
df = pd.read_csv('./dataset/hin.txt', sep = '\t', new_columns = ['english', 'hindi', 'somethingelse'])[['english','hindi']]

In [6]:
df.shape

(2908, 2)

In [7]:
df.sample(10)

english,hindi
str,str
"""She is dressed...","""उसने सफ़ेद कपड..."
"""That's his spe...","""वह उसकी ख़ासिय..."
"""We put sugar i...","""हम चाय में चीन..."
"""Turn your face...","""अपना मूँह इस ओ..."
"""I know that al...","""मैं जानता हूँ ..."
"""You can't prov...","""तुम वह साबित न..."
"""What movies ar...","""इस सप्ताह कौन ..."
"""Everyone knew ...","""वह गाना सभी को..."
"""He wiped the s...","""उसने अपने माथे..."


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
UNITS = 32
EMBEDDING_DIMS = 16
BATCH_SIZE = 32

# Prepare the data `tf.data`

In [10]:
# Split the data for train and val
train_df, val_df = train_test_split(df, test_size = .02, random_state = 4 )

In [11]:
print('train shape: ', train_df.shape)
print('val shape: ', val_df.shape)

train shape:  (2849, 2)
val shape:  (59, 2)


In [12]:
BUFFER_SIZE = 2000

In [13]:
train_ds = tf.data.Dataset.from_tensor_slices((train_df['english'].to_list(), train_df['hindi'].to_list())).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
val_ds = tf.data.Dataset.from_tensor_slices((val_df['english'].to_list(), val_df['hindi'].to_list())).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

2023-02-20 12:01:47.008042: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-20 12:01:47.008452: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-20 12:01:47.008669: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-20 12:01:48.040352: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-20 12:01:48.040586: I tensorflow/compiler/xla/stream_executo

In [14]:
len(train_ds)

90

In [15]:

# preprocessing text
def tf_lower_and_split_punct_en(text):
    # Split accented characters.
    # text = tf.text.normalize_utf8(text, 'NFKD')
    text = tf.strings.lower(text)
    # Keep space, a to z, and select punctuation.
    text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
    # Add spaces around punctuation.
    text = tf.strings.regex_replace(text, '[.?!,¿|]', r' \0 ')
    # Strip whitespace.
    text = tf.strings.strip(text)

    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')

    return text



# preprocessing text
def tf_lower_and_split_punct_hi(text):
    text = tf.strings.regex_replace(text, '[.?!,¿|]', r' \0 ')
    text = tf.strings.strip(text)
    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
    return text

In [16]:
some_hindi_text = 'उन्होंने मेरी का| मज़ाक उड़ाया'
print(some_hindi_text)
b= tf_lower_and_split_punct_hi(some_hindi_text)
print(b)
print(b.numpy().decode())
del(b)

उन्होंने मेरी का| मज़ाक उड़ाया
tf.Tensor(b'[START] \xe0\xa4\x89\xe0\xa4\xa8\xe0\xa5\x8d\xe0\xa4\xb9\xe0\xa5\x8b\xe0\xa4\x82\xe0\xa4\xa8\xe0\xa5\x87 \xe0\xa4\xae\xe0\xa5\x87\xe0\xa4\xb0\xe0\xa5\x80 \xe0\xa4\x95\xe0\xa4\xbe |  \xe0\xa4\xae\xe0\xa4\x9c\xe0\xa4\xbc\xe0\xa4\xbe\xe0\xa4\x95 \xe0\xa4\x89\xe0\xa4\xa1\xe0\xa4\xbc\xe0\xa4\xbe\xe0\xa4\xaf\xe0\xa4\xbe [END]', shape=(), dtype=string)
[START] उन्होंने मेरी का |  मज़ाक उड़ाया [END]


In [17]:
for i, j in train_ds.take(1):
    print('i: ',i)
    print('j: ', j)
    # print('j decoded: ',j.numpy().decode())

i:  tf.Tensor(
[b'The boy gathered a handful of peanuts and put them in a small box.'
 b'A madman is not accountable for his actions.' b'Tom is my friend.'
 b'Some of the apples in the box were rotten.' b'Everybody likes her.'
 b'She showed me a letter written in English.' b'Boys will be boys.'
 b'How rude of you!' b'What put such an idea into your head?'
 b'How old might his grandfather be?' b'How are you?'
 b'Bern is the capital of Switzerland.'
 b"I learned to drive a car and got a driver's license when I was eighteen."
 b"When he arrives in Tokyo, I'll call you right away."
 b"We'll cross the river in a boat."
 b"It's up to you whether to buy it or not." b'I am grateful to them.'
 b'He explained his plans in detail.' b'I explained the rule to him.'
 b"That's Tom's house with the red roof." b'The reason is very simple.'
 b'Come with us.' b'Birds fly.'
 b'They were about to leave when I arrived there.'
 b'When are you going to Europe?' b'Please throw the ball.'
 b"He's always running

# Text Vectorization


In [18]:
output_sequence_length = 200

In [19]:
eng_vectorizer = tf.keras.layers.TextVectorization(standardize = tf_lower_and_split_punct_en, output_sequence_length= output_sequence_length)
hin_vectorizer = tf.keras.layers.TextVectorization(standardize = tf_lower_and_split_punct_hi, output_sequence_length= output_sequence_length+1 )

In [20]:
# Adapting to textvectorizer
eng_vectorizer.adapt(train_ds.map(lambda x, y: x))
hin_vectorizer.adapt(train_ds.map(lambda x, y: y))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [21]:
max_token_english = len(eng_vectorizer.get_vocabulary())
max_token_hindi = len(hin_vectorizer.get_vocabulary())

print('Maxtokens:')
print( 'English : ', max_token_english)
print('Hindi: ', max_token_hindi)

Maxtokens:
English :  2359
Hindi:  3016


In [22]:
print('Text: ',some_hindi_text)
print('\nEncoded text: ,')
hin_vectorizer(some_hindi_text)

Text:  उन्होंने मेरी का| मज़ाक उड़ाया

Encoded text: ,


<tf.Tensor: shape=(201,), dtype=int64, numpy=
array([   2,  173,   40,   20, 1446,  369,    1,    3,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
     

## Mapping Vectorizer to dataset

In [23]:
BATCH_SIZE = 32

In [24]:
def make_vec(x, y ):
    x, y = eng_vectorizer(x), hin_vectorizer(y)

    y_in = y[:,:-1]
    y_out = y[:,1:]
    return (x,y_in), y_out

In [25]:
train_ds = train_ds.map(make_vec) #.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(make_vec) # .batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [26]:
for i,j in train_ds.take(1):
    print(i[1])
    print()
    print(j)
    break

tf.Tensor(
[[   2    8   57 ...    0    0    0]
 [   2 1244  117 ...    0    0    0]
 [   2   11  174 ...    0    0    0]
 ...
 [   2   12 1015 ...    0    0    0]
 [   2  152   23 ...    0    0    0]
 [   2    8  520 ...    0    0    0]], shape=(32, 200), dtype=int64)

tf.Tensor(
[[   8   57   62 ...    0    0    0]
 [1244  117 1481 ...    0    0    0]
 [  11  174   50 ...    0    0    0]
 ...
 [  12 1015  457 ...    0    0    0]
 [ 152   23   26 ...    0    0    0]
 [   8  520  172 ...    0    0    0]], shape=(32, 200), dtype=int64)


# Model

## Encoder

In [27]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, text_processor, units, embedding_dims = 32):
        super(Encoder, self).__init__()
        self.text_processor = text_processor
        self.vocab_size = text_processor.vocabulary_size()
        self.units = units
        # self.return_state = return_state
        # The embedding layer converts tokens to vectors
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, units, mask_zero=True)

        # The RNN layer processes those vectors sequentially.
        self.rnn = tf.keras.layers.Bidirectional(
            merge_mode='sum',
            layer=tf.keras.layers.LSTM(units, 
                                       return_sequences = True, 
                                       return_state = True,
                                       recurrent_initializer='glorot_uniform'))

    def call(self, x):

        # 2. The embedding layer looks up the embedding vector for each token.
        x = self.embedding(x)

        # 3. The GRU processes the sequence of embeddings.
        *x, state_h, state_c = self.rnn(x)
        tf.print('after encoder return sequence true: ',len(x) )
        state = [state_h, state_c]
    
        return x, state

    def convert_input(self, texts):
        texts = tf.convert_to_tensor(texts)
        if len(texts.shape) == 0:
            texts = tf.convert_to_tensor(texts)[tf.newaxis]
        context = self.text_processor(texts)
        context = self(context)
        return context

## Attention

In [28]:
class CrossAttention(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(key_dim=units, num_heads=1, **kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

    def call(self, x, context):

        attn_output, attn_scores = self.mha(
            query=x,
            value=context,
            return_attention_scores=True)

        # Cache the attention scores for plotting later.
        attn_scores = tf.reduce_mean(attn_scores, axis=1)
        self.last_attention_weights = attn_scores

        x = self.add([x, attn_output])
        x = self.layernorm(x)

        return x

## Decoder

In [29]:
class Decoder(tf.keras.layers.Layer):
    @classmethod
    def add_method(cls, fun):
        setattr(cls, fun.__name__, fun)
        return fun

    def __init__(self, text_processor, units, embedding_dims = 32):
        super(Decoder, self).__init__()
        self.text_processor = text_processor
        self.vocab_size = text_processor.vocabulary_size()
        self.word_to_id = tf.keras.layers.StringLookup(vocabulary=text_processor.get_vocabulary(), mask_token='', oov_token='[UNK]')
        self.id_to_word = tf.keras.layers.StringLookup(vocabulary=text_processor.get_vocabulary(), mask_token='', oov_token='[UNK]', invert=True)
        self.start_token = self.word_to_id('[START]')
        self.end_token = self.word_to_id('[END]')

        self.units = units


        # 1. The embedding layer converts token IDs to vectors
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, units, mask_zero=True)

        # 2. The RNN keeps track of what's been generated so far.
        self.rnn = tf.keras.layers.LSTM(units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

        # 3. The RNN output will be the query for the attention layer.
        self.attention = CrossAttention(units)

        # 4. This fully connected layer produces the logits for each
        # output token.
        self.output_layer = tf.keras.layers.Dense(self.vocab_size)

### Decoder call

In [30]:
@Decoder.add_method
def call(self,
         context, x,
         state=None,
         return_state=False):  

    # 1. Lookup the embeddings
    x = self.embedding(x)

    # 2. Process the target sequence.
    x = self.rnn(x, initial_state=state)
    tf.print('decoder output: ', len(x))
    # 3. Use the RNN output as the query for the attention over the context.
    x = self.attention(x, context)
    self.last_attention_weights = self.attention.last_attention_weights


    # Step 4. Generate logit predictions for the next token.
    logits = self.output_layer(x)

    if return_state:
        return logits, state
    else:
        return logits

In [31]:
@Decoder.add_method
def get_initial_state(self, context):
    batch_size = tf.shape(context)[0]
    start_tokens = tf.fill([batch_size, 1], self.start_token)
    done = tf.zeros([batch_size, 1], dtype=tf.bool)
    embedded = self.embedding(start_tokens)
    return start_tokens, done, self.rnn.get_initial_state(embedded)[0]

In [32]:
@Decoder.add_method
def tokens_to_text(self, tokens):
    words = self.id_to_word(tokens)
    result = tf.strings.reduce_join(words, axis=-1, separator=' ')
    result = tf.strings.regex_replace(result, '^ *\[START\] *', '')
    result = tf.strings.regex_replace(result, ' *\[END\] *$', '')
    return result

In [33]:
@Decoder.add_method
def get_next_token(self, context, next_token, done, state, temperature = 0.0):
    logits, state = self(context, next_token, state = state, return_state=True) 

    if temperature == 0.0:
        next_token = tf.argmax(logits, axis=-1)
    else:
        logits = logits[:, -1, :]/temperature
        next_token = tf.random.categorical(logits, num_samples=1)

    # If a sequence produces an `end_token`, set it `done`
    done = done | (next_token == self.end_token)
    # Once a sequence is done it only produces 0-padding.
    next_token = tf.where(done, tf.constant(0, dtype=tf.int64), next_token)

    return next_token, done, state

In [34]:
# class Translator(tf.keras.Model):
#     @classmethod
#     def add_method(cls, fun):
#         setattr(cls, fun.__name__, fun)
#         return fun

#     def __init__(self, units, context_text_processor, target_text_processor):
#         super().__init__()
#         # Build the encoder and decoder
#         encoder = Encoder(context_text_processor, units)
#         decoder = Decoder(target_text_processor, units)

#         self.encoder = encoder
#         self.decoder = decoder
        

#     def call(self, inputs):
#         context, x = inputs
#         tf.print('Before encoder-decoder')
#         # tf.print('inputs : ',inputs.shape)
#         tf.print('context: ',context.shape)
#         tf.print('x      : ',x.shape)
#         context = self.encoder(context)
#         tf.print()
#         logits = self.decoder(context, x)
#         tf.print('--'*20)
#         tf.print('After encoder-decoder')
#         tf.print('context: ',context.shape)
#         tf.print('logits : ',logits.shape)
#     #TODO(b/250038731): remove this
#         try:
#           # Delete the keras mask, so keras doesn't scale the loss+accuracy. 
#             del logits._keras_mask
#         except AttributeError:
#             pass

#         return logits

In [35]:
class Translator2(tf.keras.Model):
    @classmethod
    def add_method(cls, fun):
        setattr(cls, fun.__name__, fun)
        return fun

    def __init__(self, units, context_text_processor, target_text_processor):
        super().__init__()
        
        self.context_text_processor = context_text_processor
        self.target_text_processor = target_text_processor

        self.context_vocab_size = context_text_processor.vocabulary_size()
        self.target_vocab_size = target_text_processor.vocabulary_size()
        
        self.word_to_id = tf.keras.layers.StringLookup(vocabulary=target_text_processor.get_vocabulary(), mask_token='', oov_token='[UNK]')
        self.id_to_word = tf.keras.layers.StringLookup(vocabulary=target_text_processor.get_vocabulary(), mask_token='', oov_token='[UNK]', invert=True)
    
        self.units = units

        # The embedding layer converts tokens to vectors
        self.embedding1 = tf.keras.layers.Embedding(self.context_vocab_size, units, mask_zero=True)
        self.embedding2 = tf.keras.layers.Embedding(self.target_vocab_size, units, mask_zero=True)

        # The RNN layer processes those vectors sequentially.
        self.encoder = tf.keras.layers.Bidirectional(
            merge_mode='concat',
            layer=tf.keras.layers.GRU(units, 
                                       return_sequences = True, 
                                       return_state = True,
                                       recurrent_initializer='glorot_uniform'))
        
        self.decoder = tf.keras.layers.Bidirectional(
            merge_mode='concat',
            layer=tf.keras.layers.GRU(units, 
                                       return_sequences = True, 
                                       return_state = True,
                                       recurrent_initializer='glorot_uniform'))
        
        
        self.start_token = self.word_to_id('[START]')
        self.end_token = self.word_to_id('[END]')

        # 3. The RNN output will be the query for the attention layer.
        self.attention = CrossAttention(units)

        # 4. This fully connected layer produces the logits for each
        # output token.
        self.output_layer = tf.keras.layers.Dense(self.target_vocab_size)
        
        self.encoder_state = None
    def call(self, X, y=None):
        context, que = X
        
        #Encoding
        # 1. embedding
        x = self.embedding1(context)
        tf.print('after embedding: x shape: ',x.shape )
        x, enc_h, enc_c = self.encoder(x)
        self.encoder_state = [enc_h, enc_c]
        # tf.print('after encodeing: x shape: ',len(x) )

        # encoder_context, encoder_state = x
        #Decoding
        x = self.embedding1(que)
        decoder_context, decoder_state_h, decoder_state_c = self.decoder(x, initial_state=self.encoder_state)
        
        # tf.print('decoder context shape: ', len(decoder_context))
        # logits
        logits = self.output_layer(decoder_context)
            # tf.print('\n')
        #TODO(b/250038731): remove this
        try:
          # Delete the keras mask, so keras doesn't scale the loss+accuracy. 
            del logits._keras_mask
        except AttributeError:
            pass

        return logits

In [36]:
model = Translator2(UNITS, eng_vectorizer, hin_vectorizer)

In [37]:
# sample = train_df.sample(5)
# some_hindi_text = sample['hindi'].to_numpy()
# some_eng_text = sample['english'].to_numpy()

# print(some_hindi_text)
# print(some_eng_text)

# vectorized_eng_text = eng_vectorizer(some_eng_text)
# vectorized_eng_text[:,:10]

# vectorized_hindi_text = hin_vectorizer(some_hindi_text)
# vectorized_hindi_text[:,:10]

# vec_hindi_in = vectorized_hindi_text[:,:-1]
# vec_hindi_out = vectorized_hindi_text[:,1:]
# print(vec_hindi_in[:,:10])
# print()
# print(vec_hindi_out[:,:10])

# encoder = Encoder(eng_vectorizer, UNITS)
# # context_eng = encoder((vectorized_eng_text))

# new_tx = encoder.convert_input(['hey man'])

# # context_eng

# # decoder = Decoder(hin_vectorizer, UNITS)
# # logits = decoder(context_eng, vec_hindi_in)
# # logits.shape

In [38]:
def masked_loss(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    loss = loss_fn(y_true, y_pred)

    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

In [39]:
def masked_acc(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)

    match = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true != 0, tf.float32)

    return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [40]:
model.compile(optimizer='adam',
              loss=masked_loss, 
              metrics=[masked_acc, masked_loss])

In [41]:
# model.summary()

In [42]:
EPOCHS = 30
CKPT_DIR = './model_checkpoint'

In [43]:
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(CKPT_DIR,  f"{datetime.now().strftime('%m:%d:%Y, %H:%M:%S')}"),
    monitor= 'loss',
    verbose= 0,
    save_best_only = True,
    save_weights_only = True,
    mode= 'auto',
    save_freq='epoch'
)

In [44]:
# keras.utils.plot_model(model, show_shapes=True)

In [45]:
# tf.executing_eagerly(False)

history = model.fit(
    train_ds.repeat(), 
    epochs=EPOCHS,
    steps_per_epoch = 50,
    validation_data=val_ds,
    validation_steps = 20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3),
    model_ckpt])

Epoch 1/30
after embedding: x shape:  TensorShape([None, 200, 32])


2023-02-20 12:02:13.678665: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}

	while inferring type of node 'cond_43/output/_20'
2023-02-20 12:02:14.357474: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-02-20 12:02:15.947486: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:446 : INTERNAL: libdevice not found at ./libdevice.10.bc
2023-02-20 12:02:15.979367: W tensorflow/core/framework/op_kernel.cc:1830] OP_

InternalError: Graph execution error:

Detected at node 'StatefulPartitionedCall_14' defined at (most recent call last):
    File "/home/t/miniconda3/envs/tf_new/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/t/miniconda3/envs/tf_new/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/t/.local/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/t/.local/lib/python3.9/site-packages/traitlets/config/application.py", line 1041, in launch_instance
      app.start()
    File "/home/t/.local/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 724, in start
      self.io_loop.start()
    File "/home/t/miniconda3/envs/tf_new/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/t/miniconda3/envs/tf_new/lib/python3.9/asyncio/base_

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('CE/token')
plt.legend()

In [ ]:
, stateplt.plot(history.history['masked_acc'], label='accuracy')
plt.plot(history.history['val_masked_acc'], label='val_accuracy')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('CE/token')
plt.legend()

In [ ]:
@Translator.add_method
def translate(self,
              texts, *,
              max_length=50,
              temperature=0.0):
  # Process the input texts
    context = self.encoder.convert_input(texts)
    batch_size = tf.shape(texts)[0]

  # Setup the loop inputs
    tokens = []
    attention_weights = []
    next_token, done, state = self.decoder.get_initial_state(context)

    for _ in range(max_length):
        # Generate the next token
        next_token, done, state = self.decoder.get_next_token(context, next_token, done,  state, temperature)

        # Collect the generated tokens
        tokens.append(next_token)
        attention_weights.append(self.decoder.last_attention_weights)

        if tf.executing_eagerly() and tf.reduce_all(done):
            break

    # Stack the lists of tokens and attention weights.
    tokens = tf.concat(tokens, axis=-1)   # t*[(batch 1)] -> (batch, t)
    self.last_attention_weights = tf.concat(attention_weights, axis=1)  # t*[(batch 1 s)] -> (batch, t s)

    result = self.decoder.tokens_to_text(tokens)
    return result

In [ ]:
result = model.translate(['She loves me']) 
result[0].numpy().decode()

In [ ]:
plt.plot(history.history['masked_acc'], label='accuracy')
plt.ylim([0, max(plt.ylim())])
plt.title('accuracy vs epochs')
plt.legend()